## This notebook will help you train a latent Point-Cloud GAN.

(Assumes latent_3d_points is in the PYTHONPATH and that a trained AE model exists)

In [2]:
import numpy as np
import os.path as osp

from latent_3d_points.src.autoencoder import Configuration as Conf
from latent_3d_points.src.point_net_ae import PointNetAutoEncoder

from latent_3d_points.src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder

from latent_3d_points.src.tf_utils import reset_tf_graph

from latent_3d_points.src.vanilla_gan import Vanilla_GAN
from latent_3d_points.src.w_gan_gp import W_GAN_GP

from latent_3d_points.src.generators_discriminators import latent_code_discriminator_two_layers,\
latent_code_generator_two_layers

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
# Set DATA/AE parameters

top_out_dir = '../data/'                                       # Use to save Neural-Net check-points etc.
top_in_dir = '../data/shape_net_core_uniform_samples_2048/'    # Top-dir of where point-clouds are stored.

ae_configuration = '../data/single_class_ae/configuration'     # AE model-description. You can alternatively, 
                                                               # use your own way to load a pre-trained AE.

ae_epoch = 100                                                 # Model/epoch of AE to load.
bneck_size = 128                                               # Bottleneck-AE size

experiment_name = 'latent_gan'
n_pc_points = 2048                                             # Number of points per model.

class_name = raw_input('Give me the class name (e.g. "chair"): ').lower()

Give me the class name (e.g. "chair"): chair


In [ ]:
# Set GAN, training parameters.

# save_model = False
# class_name = raw_input('Give me the class type.\n').lower()
# syn_id = snc_category_to_synth_id()[class_name]
# ae_loss = 'emd'
use_wgan = False
max_epochs = 500
plot_train_curve = True
saver_step = np.hstack([np.array([1, 5, 10]), np.arange(50, max_epochs+1, 50)])
save_synthetic_samples = True
experiment_tag = 'test'

init_lr = 0.0001
batch_size = 50
noise_params = {'mu':0, 'sigma': 0.2}
noise_dim = bneck_size
beta = 0.5
n_syn_samples = train_data.num_examples  # How many samples to produce in each save step.
n_out = [bneck_size]
accum_syn_data = []
train_stats = []

if save_synthetic_samples:
    synthetic_data_out_dir = osp.join(top_out_dir, 'OUT/synthetic_samples/', experiment_tag)
    create_dir(synthetic_data_out_dir)

In [5]:
# Load point-clouds
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)

6778 pclouds were loaded. They belong in 1 shape-classes.


In [6]:
# Load pre-trained AE
reset_tf_graph()
ae_conf = Conf.load(ae_configuration)
ae = PointNetAutoEncoder(ae_conf.experiment_name, ae_conf)
ae.restore_model(ae_conf.train_dir, ae_epoch, verbose=True)

Building Encoder
encoder_conv_layer_0 conv params =  256 bnorm params =  128
Tensor("single_class_ae_2/Relu:0", shape=(?, 2048, 64), dtype=float32)
output size: 131072 

encoder_conv_layer_1 conv params =  8320 bnorm params =  256
Tensor("single_class_ae_2/Relu_1:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_2 conv params =  16512 bnorm params =  256
Tensor("single_class_ae_2/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_3 conv params =  33024 bnorm params =  512
Tensor("single_class_ae_2/Relu_3:0", shape=(?, 2048, 256), dtype=float32)
output size: 524288 

encoder_conv_layer_4 conv params =  32896 bnorm params =  256
Tensor("single_class_ae_2/Relu_4:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

Tensor("single_class_ae_2/Max:0", shape=(?, 128), dtype=float32)
Building Decoder
decoder_fc_0 FC params =  33024 Tensor("single_class_ae_2/Relu_5:0", shape=(?, 256), dtype=float32)
output size: 256 

In [19]:
# Convert raw-data to latent codes.
latent_codes = ae.get_latent_codes(all_pc_data.point_clouds)
train_data = PointCloudDataSet(latent_codes)

In [19]:
reset_tf_graph()

if use_wgan:
    lam = 10
    gan = W_GAN_GP(experiment_tag, init_lr, lam, n_out, noise_dim, \
                  latent_code_discriminator_two_layers, 
                  latent_code_generator_two_layers,\
                  beta=beta
                 )
else:    
    gan = Vanilla_GAN(experiment_tag, init_lr, n_out, noise_dim,
                     latent_code_discriminator_two_layers, latent_code_generator_two_layers,
                     beta=beta
                    )

In [19]:
# Train the GAN.
for _ in range(max_epochs):
    loss, duration = gan._single_epoch_train(train_data, batch_size, noise_params)
    epoch = int(gan.sess.run(gan.epoch.assign_add(tf.constant(1.0))))
    print epoch, loss

    if save_model and (epoch % saver_step == 0 or epoch <= 5):
        checkpoint_path = osp.join(train_dir, MODEL_SAVER_ID)
        gan.saver.save(gan.sess, checkpoint_path, global_step=gan.epoch)

    if save_synthetic_samples and epoch in saver_step:
        syn_latent_data = gan.generate(n_syn_samples, noise_params)
        syn_data = ae.decode(syn_latent_data)
        np.savez(osp.join(synthetic_data_out_dir, 'epoch_' + str(epoch)), syn_data)
        for k in range(3):
            Point_Cloud(syn_data[k]).plot()

    train_stats.append((epoch,) + loss)

1 (0.41234670566475912, 1.9826304430546968)
2 (0.092744595168725311, 3.7401423350624414)
3 (0.086751907124467523, 4.9079365854677945)
4 (0.14127983541592307, 5.5437571193860924)
5 (0.11072289509617764, 6.0356144200200621)
6 (0.057010742665632913, 6.0249628647514015)
7 (0.034189508660979896, 6.2511722398840863)
8 (0.02702255496836227, 6.6130070810732633)
9 (0.028049864069275236, 6.6683759316154152)
10 (0.039642552173008092, 6.2399310899817424)
11 (0.067302263495714762, 5.3281164915665338)
12 (0.095872920589602506, 4.6584717170051908)
13 (0.11670367102260175, 4.179448787025783)
14 (0.13206631431113119, 3.9878793301789659)
15 (0.16565291175375815, 3.8262052411618441)
16 (0.2338887218547904, 3.6161000645678976)
17 (0.31642804664114249, 3.3541466132454252)
18 (0.4007429537565812, 3.000937961495441)
19 (0.50492612304894824, 2.6533821873042895)
20 (0.60389162457507595, 2.2986403237218442)
21 (0.69011772767357205, 2.038876848635466)
22 (0.76728998733603437, 1.8087486132331516)
23 (0.8442868419

In [20]:
if plot_train_curve:
    x = range(len(train_stats))
    d_loss = [t[1] for t in train_stats]
    g_loss = [t[2] for t in train_stats]
    plt.plot(x, d_loss, '--')
    plt.plot(x, g_loss)
    plt.title('Latent GAN training. (%s, %s)' %(class_name, ae_loss))
    plt.legend(['Discriminator', 'Generator'], loc=0)
    
    plt.tick_params(axis='x', which='both', bottom='off', top='off')
    plt.tick_params(axis='y', which='both', left='off', right='off')
    
    plt.xlabel('Epochs.') 
    plt.ylabel('Loss.')

In [ ]:
* Εχεις κρατησει τα txts για του περισσοτερους AEs?
* You can manually read them and write them to the appropriate functions and then also 
add some reading/creation function from raw txt for the configuration.





# # TEMP
# TO load work with pre-trained SN models.
# import sys
# sys.path.append("../../../../Git_Repos/")
# from research.iclr.helper import load_multiple_version_of_pcs
# # in_data = load_multiple_version_of_pcs('uniform_one', syn_id, n_classes=1)
# # train_data = in_data['train']
# !cat /orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/ae_chair_mlp_with_split_1pc_usampled_bnorm_on_encoder_only_2048_pts_128_bneck_emd/configuration.txt

# ae_configuration = '/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/ae_chair_mlp_with_split_1pc_usampled_bnorm_on_encoder_only_2048_pts_128_bneck_emd/configuration.txt

ae_conf = Conf.load(ae_configuration)

# saved_epochs = read_saved_epochs(ae_conf.train_dir)
# _, best_epoch = find_best_validation_epoch_from_train_stats(osp.join(ae_train_dir, 'train_stats.txt'))
# if best_epoch % ae_conf.saver_step != 0: # Model was not saved at that epoch.
#     best_epoch += best_epoch % ae_conf.saver_step
# ae_conf.encoder_args['verbose'] = False
# ae_conf.decoder_args['verbose'] = False
# reset_tf_graph()
# ae = PointNetAutoEncoder(ae_conf.experiment_name, ae_conf)    
# ae.restore_model(ae_conf.train_dir, best_epoch, verbose=True)